# ModelAnimation Example Notebook

This sample notebook demos the use of ModelAnimation on a simple TF model.

The first few cells have nothing to do with ModelAnimation, other than setting up the model.

In [ ]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

from sklearn.model_selection import train_test_split
import numpy as np
import random
import matplotlib.pyplot as plt

Creating a simple 'home-made' data set.

In [ ]:
template = np.array([[2.0,2.0,2.0,0.1,0.1,0.1,0.1,0.1,0.1],
                     [0.1,0.1,2.0,0.1,0.1,2.0,0.1,0.1,2.0],
                     [0.1,0.1,0.1,0.1,0.1,0.1,2.0,2.0,2.0],
                     [2.0,0.1,0.1,2.0,0.1,0.1,2.0,0.1,0.1],
                     [2.0,0.1,0.1,0.1,2.0,0.1,0.1,0.1,2.0],
                     [0.1,0.1,2.0,0.1,2.0,0.1,2.0,0.1,0.1]])

In [ ]:
X = []
y = []

for _ in range(2000):
    for i in range(len(template)):
        r = np.random.rand(9)
        X.append(template[i] * r)
        y.append(i)
        
X = np.array(X)
y = np.array(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [ ]:
plt.figure(figsize=(15,5))
for i in range(16*4):
    plt.subplot(4,16,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(X_train[i].reshape(3,3), cmap='Greys')
    plt.xlabel(y_train[i])
plt.show()

# ModelAnimation

## The following cell includes the ModelAnimation code, and sets up a custom TensorFlow Keras callback.  There are three callbacks:

- on_train_begin - This stores the model weights in a list called `model_weights` at the start of the training.  Boradly speaking this will store the randomised starting position.

- on_epoch_end - This will append to `model_weights` after each epoch

- on_train_end - When the training is complete, this callback will trigger the rendering of the frames and optionally the animation.  When calling `create_animation` there are several named parameters you can pass in.  These are listed inthe readme file in github. 

In [ ]:
from ModelAnimation import ModelAnimation

class CustomCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        model_weights.append(model.get_weights())
        
    def on_epoch_end(self, epoch, logs=None):
        model_weights.append(model.get_weights())

    def on_train_end(self, logs=None):
        animation = ModelAnimation()
        animation.create_animation(model_weights, model.input.shape.as_list(),
                                   margin=150,
                                   node_size=50,
                                   node_gap=20,
                                   conn_max_width=10,
                                   background_rgba=(220,220,220,255),
                                   gif=True, 
                                   frame_numbers=True)

## At the start of this cell we create `model_weights`.  We do this here so that if we re-run the training, the list will start again.

In [ ]:
# Clear some value (incase we run multiple times)
tf.keras.backend.clear_session()
model_weights = []

# Create a TF sequential model with Keras
model = tf.keras.models.Sequential([
  tf.keras.layers.Input(9),
  tf.keras.layers.Dense(12, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## When we call `model.fit` we pass in the `callbacks` object, and the animation will run automatically at the end of the training.

In [ ]:
e = model.fit(X_train, y_train,
              epochs=10,
              callbacks=[CustomCallback()])